# Implementation of Intriguing Properties of NN (TensorFlow Implementation)

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function


# Import initial dependancies
import numpy as np
import tensorflow as tf

tf.enable_eager_execution()

In [2]:
import matplotlib.pyplot as plt
%matplotlib inline

### Global vars

In [3]:
weight_decay = 0
num_epochs = 10
batch_size = 128

### Data Prep 

In [4]:
# mnist_train_ds, mnist_test_ds = datasets['train'], datasets['test']
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# Use tf.data to batch and shuffle the ds
mnist_train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(1024).batch(32)

mnist_test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(batch_size)

### Define a FC model

In [5]:
from model import FC
fc_model = FC()

### Define a Loss and Optimizer

In [6]:
# Since there are 10 different classes in this ds, define loss as categorical
loss_func = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

# Set the metrics to measure the loss and accuracy of the model
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')


### Model training

In [7]:
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        predictions = fc_model(images)
        loss = loss_func(labels, predictions)
    gradients = tape.gradient(loss, fc_model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, fc_model.trainable_variables))
    
    train_loss(loss)
    train_accuracy(labels, predictions)

In [8]:
@tf.function
def test_step(images, labels):
    predictions = fc_model(images)
    t_loss = loss_func(labels, predictions)
    
    test_loss(t_loss)
    test_accuracy(labels, predictions)

In [9]:
# Train the model
for epoch in range(num_epochs):
    # Reset the metrics at the start of each epoch
    train_loss.reset_states()
    train_accuracy.reset_states()
    
    test_loss.reset_states()
    test_accuracy.reset_states()
    
    
    
    for images, labels in mnist_train_ds:
        train_step(images, labels)
        
    for test_images, test_labels in mnist_test_ds:
        test_step(test_images, test_labels)
        
    template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
    print(template.format(epoch+1,
                          train_loss.result(),
                          train_accuracy.result() * 100,
                          test_loss.result(),
                          test_accuracy.result() * 100))
    

Epoch 1, Loss: 0.25813350081443787, Accuracy: 92.63166809082031, Test Loss: 0.1509578377008438, Test Accuracy: 95.38999938964844
Epoch 2, Loss: 0.11316273361444473, Accuracy: 96.64166259765625, Test Loss: 0.11231496185064316, Test Accuracy: 96.45000457763672
Epoch 3, Loss: 0.07703680545091629, Accuracy: 97.70667266845703, Test Loss: 0.10229221731424332, Test Accuracy: 96.87999725341797
Epoch 4, Loss: 0.056369755417108536, Accuracy: 98.31500244140625, Test Loss: 0.0951400175690651, Test Accuracy: 97.0999984741211
Epoch 5, Loss: 0.04166274517774582, Accuracy: 98.80833435058594, Test Loss: 0.09396157413721085, Test Accuracy: 97.12999725341797
Epoch 6, Loss: 0.03182442486286163, Accuracy: 99.11666870117188, Test Loss: 0.08917468786239624, Test Accuracy: 97.33000183105469
Epoch 7, Loss: 0.02418573945760727, Accuracy: 99.34333801269531, Test Loss: 0.1008484959602356, Test Accuracy: 97.18000030517578
Epoch 8, Loss: 0.019030239433050156, Accuracy: 99.48500061035156, Test Loss: 0.09840416908264

In [10]:
fc_model.summary()

Model: "fc"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential (Sequential)      multiple                  101770    
Total params: 101,770
Trainable params: 101,770
Non-trainable params: 0
_________________________________________________________________


# Units of  *φ(x)*

*x′ = arg max⟨φ(x), J⟩*, where *J* is the data that was not used for training

In [130]:

for i in range(5):
    unit_vec = tf.eye(128)[i,:]
    
    for data in mnist_test_ds.batch(3):
        for image in data[0]:
            x = tf.cast(image, 'float32')        
            if (x.shape[1] != batch_size):
                break
            values = tf.multiply(x, unit_vec)
        
    
    # TODO: Display the images with the vector projections
    
    
        
    

In [125]:
def show_image(image):
    plt.figure(figsize=(5, 20))
    plt.imshow(image)
    plt.show()